In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-06-15 16:52:46.590445: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 16:52:46.631233: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-15 16:52:46.631952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 16:52:50.057037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def npz_to_tensor(loc:str):
    data = np.load(loc)
    
    array1 = data['foreground']
    tensor1 = tf.convert_to_tensor(array1)
    
    if 'label' in data:
        array2 = data['label']
        tensor2 = tf.convert_to_tensor(array2,dtype=tf.int32)
        return tensor1, tensor2
    
    return tensor1


In [3]:
X_train, Y_train = npz_to_tensor('data/classification_training_data.npz')
X_val, Y_val = npz_to_tensor('data/classification_validation_data.npz')
X_test = npz_to_tensor('data/classification_test_data.npz')

In [10]:
# Define the model architecture
model = tf.keras.Sequential()
model.add(layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(2048, 1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 2046, 32)          128       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1023, 32)         0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 32736)             0         
                                                                 
 dense_9 (Dense)             (None, 128)               4190336   
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                      

In [14]:
model.fit(x=X_train,
              y=Y_train,
              epochs=10,
              verbose=1,
              batch_size=32,
              validation_data=(X_val, Y_val),
              callbacks=[
                  EarlyStopping(monitor='val_loss', patience=5),
                  ReduceLROnPlateau(verbose=1, patience=2, monitor='val_loss')
              ])

Epoch 1/10
313/313 [==============================] - 28s 79ms/step - loss: 7.1076 - accuracy: 0.3550 - val_loss: 1.1053 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 2/10
313/313 [==============================] - 24s 78ms/step - loss: 1.1044 - accuracy: 0.3683 - val_loss: 1.1055 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 3/10
313/313 [==============================] - 25s 80ms/step - loss: 1.2710 - accuracy: 0.3694 - val_loss: 1.0986 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 4/10
313/313 [==============================] - 24s 76ms/step - loss: 1.1030 - accuracy: 0.3685 - val_loss: 1.0985 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 5/10
313/313 [==============================] - 24s 78ms/step - loss: 1.0955 - accuracy: 0.3688 - val_loss: 1.0984 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 6/10
313/313 [==============================] - 24s 78ms/step - loss: 1.0956 - accuracy: 0.3686 - val_loss: 1.0982 - val_accuracy: 0.3465 - lr: 0.0010
Epoch 7/10
313/313 [==============================] - 25s 